In [ ]:
#extracting only lang=en from full_dataset to full_dataset_clean_en
tsv_file = open(r"C:\Users\alice\Downloads\full_dataset_clean.tsv\full_dataset_clean.tsv", 'r+')
tsv_file_en = open(r"C:\Users\alice\Downloads\full_dataset_clean.tsv\full_dataset_clean_en.tsv", 'a+')
df = csv.reader(tsv_file, delimiter="\t")

counter = 0
for line in df:
    if counter % 10000000 == 0:
        print(counter, '/316800308')
    if line[3] == 'en':
        tsv_file_en.write('\t'.join(line)+'\n')
    counter += 1

In [ ]:
from textblob import TextBlob
import tweepy 
import csv

#read the full_dataset_clean_en and create a test_en_sentiment file to append sentiment score
tsv_file_en = open(r"C:\Users\alice\Downloads\full_dataset_clean.tsv\full_dataset_clean_en.tsv", 'r+') 
tsv_file_en_sentiment = open(r"C:\Users\alice\Downloads\full_dataset_clean.tsv\test_en_sentiment.tsv", 'a+')
df = csv.reader(tsv_file_en, delimiter="\t")

#our Twitter API key and token
consumer_key = "aODPYB1Gbw8dyCgACFcAwO6wI"
consumer_key_secret = "F08P3xpxCbhKy0a2cxjM0b1HjV6qzSDIvdPRVKBrq09YI0PA79"
access_token = "1429992676665958446-8ewNMBPjUzZHaIfCEBuN2NQcThg2aH"
access_token_secret = "IVuX29eSqQUDW98tz3AeqspLJRbkP6oPKHjKNUFbR2b6v"

#accessing tweet
auth = tweepy.OAuthHandler(consumer_key, consumer_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

#read tweeter id,calculate polarity score and subjectivity score by Textblob, and write into test_en_sentiment
counter = 0
for line in df:
    if counter % 100000 == 0:         #keep track of downloading progress
        print(counter, '/1500000')
    try:
        tweetFetched = api.get_status(line[0])
        testimonial = TextBlob(tweetFetched.text)
        line += [str(testimonial.sentiment[0]), str(testimonial.sentiment[1])]
    except:
        line += ['nan', 'nan']
        
    tsv_file_en_sentiment.write('\t'.join(line)+'\n')
    counter += 1

In [9]:
import pandas as pd
import numpy as np

# Open the crawled twitter file with sentiment scores, turn it into a data structure to add column and manipulate data 
polarity_df= pd.read_csv('test_en_sentiment.tsv', delimiter= "\t", header=None)
polarity_df.columns= ["Tweet ID", "Date", "Time", "Language", "NULL", "Polarity", "Subjectivity"]
polarity_df

,Tweet ID,Date,Time,Language,NULL,Polarity,Subjectivity
0,1213330173736738817,2020-01-04,05:23:50,en,NaN,0.216667,0.450000
1,1214195710301618178,2020-01-06,14:43:10,en,NaN,NaN,NaN
2,1214198629553950721,2020-01-06,14:54:46,en,NaN,-0.121212,0.484848
3,1214917256414662656,2020-01-08,14:30:20,en,NaN,0.000000,0.000000
4,1214936881563082752,2020-01-08,15:48:19,en,NaN,NaN,NaN
...,...,...,...,...,...,...,...
719156,1223395516446167040,2020-01-31,23:59:55,en,NaN,NaN,NaN
719157,1223395524138631173,2020-01-31,23:59:57,en,NaN,NaN,NaN
719158,1223395526223118336,2020-01-31,23:59:57,en,NaN,NaN,NaN
719159,1223395530702761984,2020-01-31,23:59:58,en,NaN,NaN,NaN


In [10]:
#extracting only the date and polarity score
polarity_df= polarity_df[["Date", "Polarity"]]
polarity_df

,Date,Polarity
0,2020-01-04,0.216667
1,2020-01-06,NaN
2,2020-01-06,-0.121212
3,2020-01-08,0.000000
4,2020-01-08,NaN
...,...,...
719156,2020-01-31,NaN
719157,2020-01-31,NaN
719158,2020-01-31,NaN
719159,2020-01-31,NaN


In [12]:
#filtering out polarity that are "nan"
filtered_df = polarity_df[polarity_df["Polarity"].notnull()]
filtered_df

,Date,Polarity
0,2020-01-04,0.216667
2,2020-01-06,-0.121212
3,2020-01-08,0.000000
5,2020-01-08,0.136364
6,2020-01-08,0.090909
...,...,...
716960,2020-01-31,0.000000
716961,2020-01-31,0.136364
716962,2020-01-31,0.000000
716963,2020-01-31,0.136364


In [18]:
#create a file with the filtered out individual polarity
filtered_df.to_csv('Final_individual_polarity_dataset.csv')